In [55]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
from paperswithcode import PapersWithCodeClient
client = PapersWithCodeClient()

In [3]:
methods = [
    meth for attr in dir(client) 
    if not attr.startswith('_') 
    and callable(meth := getattr(client, attr))
]

In [4]:
len(methods)

56

In [5]:
[meth.__name__ for meth in methods]
[meth.__name__ for meth in methods if "evaluation" in meth.__name__]

['dataset_evaluation_list',
 'evaluation_create',
 'evaluation_delete',
 'evaluation_get',
 'evaluation_list',
 'evaluation_metric_add',
 'evaluation_metric_delete',
 'evaluation_metric_get',
 'evaluation_metric_list',
 'evaluation_metric_update',
 'evaluation_result_add',
 'evaluation_result_delete',
 'evaluation_result_get',
 'evaluation_result_list',
 'evaluation_result_update',
 'evaluation_synchronize',
 'evaluation_update',
 'task_evaluation_list']

In [6]:
a_eval_table = client.evaluation_list(page=1, items_per_page=1)

In [7]:
neval_tables = a_eval_table.count
print(f"{neval_tables=}")

neval_tables=8557


In [32]:
import time
import pickle
from pathlib import Path

TMP_DIR = Path.cwd() / "tmp"
TMP_DIR.mkdir(exist_ok=True)

EVAL_TABLES_RECORDS_FPATH = TMP_DIR / "evaluation-tables-records.pkl"

if EVAL_TABLES_RECORDS_FPATH.is_file():
    
    with EVAL_TABLES_RECORDS_FPATH.open("rb") as f:
        eval_tables_records = pickle.load(f)
        
else:

    all_eval_tables = []

    page_idx = 1

    while True:
        
        print(f"{page_idx=}")
        
        tables_response = client.evaluation_list(page=page_idx, items_per_page=500)
        all_eval_tables.extend(tables_response.results)

        if tables_response.next_page is None:
            break

        page_idx += 1
        time.sleep(1)
        
    print(f"{len(all_eval_tables)=}")

    all_eval_tables_records = [
        dict(
            id=table.id,
            task=table.task,
            dataset=table.dataset,
        )
        for table in all_eval_tables
    ]

    with EVAL_TABLES_RECORDS_FPATH.open("wb") as f:
        pickle.dump(all_eval_tables_records, f)

page_idx=1
page_idx=2
page_idx=3
page_idx=4
page_idx=5
page_idx=6
page_idx=7
page_idx=8
page_idx=9
page_idx=10
page_idx=11
page_idx=12
page_idx=13
page_idx=14
page_idx=15
page_idx=16
page_idx=17
page_idx=18
len(all_eval_tables)=8728


In [34]:
import pandas as pd
df = pd.DataFrame.from_records(eval_tables_records)
df.describe()

,id,task,dataset
count,8557,8557,8557
unique,6626,1784,4830
top,link-prediction-on-abstrct-neoplasm,speech-recognition,big-bench
freq,4,287,154


In [35]:
TASK_ANOMALY_DETECTION = "anomaly-detection"
# DATASET_MVTECAD = "mvtecad"

In [36]:
df[df["task"] == TASK_ANOMALY_DETECTION]["dataset"].unique()

array(['dior', 'surface-defect-saliency-of-magnetic-tile',
       'assira-cat-vs-dog', 'one-class-cifar-10', 'one-class-imagenet-30',
       'unlabeled-cifar-10-vs-cifar-100', 'census',
       'numenta-anomaly-benchmark', 'road-anomaly', 'ucsd-peds2',
       'fishyscapes-l-f', 'fishyscapes', 'mvtec-3d-ad', 'stl-10', 'ucsd',
       'one-class-cifar-100', 'thyroid', 'fashion-mnist', 'shanghaitech',
       'chuk-avenue', 'anomaly-detection-on-unlabeled-cifar-10-vs',
       'anomaly-detection-on-unlabeled-imagenet-30-vs',
       'anomaly-detection-on-anomaly-detection-on', 'lost-and-found',
       'unsw-nb15-analysis', 'cardio', 'unsw-nb15-backdoor', 'mnist',
       'ag-news'], dtype=object)

In [37]:
[s for s in df["dataset"].unique() if "mvtec" in s]

['mvtec-3d-ad']

In [58]:
from paperswithcode import PapersWithCodeClient
client = PapersWithCodeClient()

In [59]:
ds_mvtecad = client.dataset_get("mvtecad")
ds_mvtecad
client.dataset_evaluation_list(dataset_id=ds_mvtecad.id)

Dataset(id='mvtecad', name='MVTecAD', full_name='MVTEC ANOMALY DETECTION DATASET', url='https://www.mvtec.com/company/research/datasets/mvtec-ad/')

EvaluationTables(count=0, next_page=None, previous_page=None, results=[])

In [60]:
ds_mvtec_ad = client.dataset_get("mvtec-ad")
ds_mvtec_ad
client.dataset_evaluation_list(dataset_id=ds_mvtec_ad.id)

Dataset(id='mvtec-ad', name='MVTec AD', full_name=None, url=None)

EvaluationTables(count=2, next_page=None, previous_page=None, results=[EvaluationTable(id='anomaly-detection-on-mvtec-ad', task='anomaly-detection', dataset='mvtec-ad', description='', mirror_url=None), EvaluationTable(id='few-shot-anomaly-detection-on-mvtec-ad', task='few-shot-anomaly-detection', dataset='mvtec-ad', description='', mirror_url=None)])

In [63]:
TASK_ANOMALY_DETECTION = "anomaly-detection"
DATASET_MVTECAD = "mvtec-ad"

def eval_table_id(task: str, dataset: str) -> str:
    return f"{task}-on-{dataset}"

In [64]:
et_ad_mvtecad = client.evaluation_get(eval_table_id(TASK_ANOMALY_DETECTION, DATASET_MVTECAD))

In [65]:
et_ad_mvtecad

EvaluationTable(id='anomaly-detection-on-mvtec-ad', task='anomaly-detection', dataset='mvtec-ad', description='', mirror_url=None)